In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the dataset
data_path = 'C:\\Users\\Gideon Price\\Downloads/Housing.csv'
housing = pd.read_csv(data_path)
housing.head()

# Preprocessing data
def binary(x):
    return x.map({'no' : 0, 'yes' : 1})

columns_to_convert = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'prefarea', 'airconditioning']

for column in columns_to_convert:
    housing[column] = binary(housing[column])
housing = housing.drop('furnishingstatus', axis=1)
# Splitting the data into features and target
x = housing[['area', 'bedrooms', 'bathrooms', 'stories', 'parking','mainroad', 'guestroom', 'basement', 'guestroom', 'hotwaterheating', 'basement', 'prefarea', 'airconditioning']].values
y = housing['price'].values


In [2]:
housing.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea
0,13300000,7420,4,2,3,1,0,0,0,1,2,1
1,12250000,8960,4,4,4,1,0,0,0,1,3,0
2,12250000,9960,3,2,2,1,0,1,0,0,2,1
3,12215000,7500,4,2,2,1,0,1,0,1,3,1
4,11410000,7420,4,1,2,1,1,1,0,1,2,0


In [3]:
# 80/20 Train/Test split
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.8, test_size=0.2, random_state=42)

# Further split the training set into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.8, test_size=0.2, random_state=42)

# Normalizing the input data
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
x_val = scaler.transform(x_val)

# Normalize output data
y_train = scaler.fit_transform(y_train.reshape(-1, 1)).flatten()
y_test = scaler.fit_transform(y_test.reshape(-1, 1)).flatten()
y_val = scaler.transform(y_val.reshape(-1, 1)).flatten()


# Convert data to PyTorch tensors
x_train = torch.tensor(x_train, dtype=torch.float32)
x_test = torch.tensor(x_test, dtype=torch.float32)
x_val = torch.tensor(x_val, dtype=torch.float32)
y_train =  torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)



In [4]:
# Define the neural network model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.hidden = nn.Linear(x_train.shape[1], 32)
        self.output = nn.Linear(32, 1)
    
    def forward(self, x):
        x = torch.relu(self.hidden(x))
        x = self.output(x)
        return x

model = NeuralNetwork()


In [5]:
# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 5000
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    train_outputs = model(x_train)
    train_loss = criterion(train_outputs, y_train)
    train_loss.backward()
    optimizer.step()
    
    model.eval()
    with torch.no_grad():
        val_outputs = model(x_val)
        val_loss = criterion(val_outputs, y_val)
    
    if (epoch + 1) % 500 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {train_loss.item():.4f}, Validation Loss: {val_loss.item():.4f}')

# Evaluate the model
model.eval()
with torch.no_grad():
    test_outputs = model(x_test)
    test_loss = criterion(test_outputs, y_test)
    print(f'Test Loss: {test_loss.item():.4f}')

# Make predictions
with torch.no_grad():
    predictions = model(x_test)
    print(predictions)

C:\Users\Gideon Price\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([348])) that is different to the input size (torch.Size([348, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\Gideon Price\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([88])) that is different to the input size (torch.Size([88, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [500/5000], Training Loss: 1.0001, Validation Loss: 0.6131
Epoch [1000/5000], Training Loss: 1.0000, Validation Loss: 0.6132
Epoch [1500/5000], Training Loss: 1.0000, Validation Loss: 0.6132
Epoch [2000/5000], Training Loss: 1.0000, Validation Loss: 0.6132
Epoch [2500/5000], Training Loss: 1.0000, Validation Loss: 0.6132
Epoch [3000/5000], Training Loss: 1.0000, Validation Loss: 0.6132
Epoch [3500/5000], Training Loss: 1.0000, Validation Loss: 0.6132
Epoch [4000/5000], Training Loss: 1.0000, Validation Loss: 0.6131
Epoch [4500/5000], Training Loss: 1.0000, Validation Loss: 0.6131
Epoch [5000/5000], Training Loss: 1.0000, Validation Loss: 0.6131
Test Loss: 1.0002
tensor([[ 2.6534e-04],
        [ 1.9009e-04],
        [-6.3489e-04],
        [-2.5724e-04],
        [ 7.6701e-04],
        [ 8.6565e-03],
        [ 1.0800e-03],
        [-7.9372e-03],
        [-4.2408e-02],
        [ 7.1701e-04],
        [ 1.4599e-02],
        [-4.6885e-04],
        [-1.0195e-02],
        [-6.4208e-04],
 

C:\Users\Gideon Price\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([109])) that is different to the input size (torch.Size([109, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [4]:
# Reload the dataset
data_path = 'C:\\Users\\Gideon Price\\Downloads/Housing.csv'
housing = pd.read_csv(data_path)
housing.head()

# Preprocessing data
def binary(x):
    return x.map({'no' : 0, 'yes' : 1})

columns_to_convert = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'prefarea', 'airconditioning']

for column in columns_to_convert:
    housing[column] = binary(housing[column])
housing = housing.drop('furnishingstatus', axis=1)
# Splitting the data into features and target
x = housing[['area', 'bedrooms', 'bathrooms', 'stories', 'parking','mainroad', 'guestroom', 'basement', 'guestroom', 'hotwaterheating', 'basement', 'prefarea', 'airconditioning']].values
y = housing['price'].values

In [6]:
# 80/20 Train/Test split
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.8, test_size=0.2, random_state=42)

# Further split the training set into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.8, test_size=0.2, random_state=42)

# Normalizing the input data
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
x_val = scaler.transform(x_val)

# Normalize output data
y_train = scaler.fit_transform(y_train.reshape(-1, 1)).flatten()
y_test = scaler.fit_transform(y_test.reshape(-1, 1)).flatten()
y_val = scaler.transform(y_val.reshape(-1, 1)).flatten()


# Convert data to PyTorch tensors
x_train = torch.tensor(x_train, dtype=torch.float32)
x_test = torch.tensor(x_test, dtype=torch.float32)
x_val = torch.tensor(x_val, dtype=torch.float32)
y_train =  torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)

In [7]:
# Add two more layers
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.hidden1 = nn.Linear(x_train.shape[1], 32)
        self.hidden2 = nn.Linear(32, 64)
        self.hidden3 = nn.Linear(64, 16)
        self.output = nn.Linear(16, 1)
    
    def forward(self, x):
        x = torch.relu(self.hidden1(x))
        x = torch.relu(self.hidden2(x))
        x = torch.relu(self.hidden3(x))
        x = self.output(x)
        return x

model = NeuralNetwork()

In [8]:
# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 5000
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    train_outputs = model(x_train)
    train_loss = criterion(train_outputs, y_train)
    train_loss.backward()
    optimizer.step()
    
    model.eval()
    with torch.no_grad():
        val_outputs = model(x_val)
        val_loss = criterion(val_outputs, y_val)
    
    if (epoch + 1) % 500 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {train_loss.item():.4f}, Validation Loss: {val_loss.item():.4f}')

# Evaluate the model
model.eval()
with torch.no_grad():
    test_outputs = model(x_test)
    test_loss = criterion(test_outputs, y_test)
    print(f'Test Loss: {test_loss.item():.4f}')

# Make predictions
with torch.no_grad():
    predictions2 = model(x_test)
    print(predictions2)

C:\Users\Gideon Price\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([348])) that is different to the input size (torch.Size([348, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\Gideon Price\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([88])) that is different to the input size (torch.Size([88, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [500/5000], Training Loss: 1.0000, Validation Loss: 0.6127
Epoch [1000/5000], Training Loss: 1.0000, Validation Loss: 0.6127
Epoch [1500/5000], Training Loss: 1.0000, Validation Loss: 0.6127
Epoch [2000/5000], Training Loss: 1.0000, Validation Loss: 0.6127
Epoch [2500/5000], Training Loss: 1.0000, Validation Loss: 0.6127
Epoch [3000/5000], Training Loss: 1.0000, Validation Loss: 0.6127
Epoch [3500/5000], Training Loss: 1.0000, Validation Loss: 0.6126
Epoch [4000/5000], Training Loss: 1.0000, Validation Loss: 0.6127
Epoch [4500/5000], Training Loss: 1.0000, Validation Loss: 0.6127
Epoch [5000/5000], Training Loss: 1.0000, Validation Loss: 0.6127
Test Loss: 1.0000
tensor([[ 1.3656e-03],
        [ 1.2268e-04],
        [ 1.5344e-05],
        [ 1.3089e-04],
        [-8.5682e-06],
        [ 1.1229e-04],
        [-1.8464e-04],
        [ 5.1699e-04],
        [ 2.1157e-03],
        [-6.2138e-06],
        [-5.3014e-04],
        [ 7.0445e-06],
        [ 8.2085e-04],
        [-4.2971e-04],
 

C:\Users\Gideon Price\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([109])) that is different to the input size (torch.Size([109, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [5]:
'''Question 2'''
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, f1_score

# Transformations for data normalization
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainloader = DataLoader(trainset, batch_size=100, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


Files already downloaded and verified
Files already downloaded and verified


In [3]:
class FullyConnectedNN(nn.Module):
    def __init__(self):
        super(FullyConnectedNN, self).__init__()
        self.flatten = nn.Flatten()
        self.hidden = nn.Linear(3 * 32 * 32, 512)  # CIFAR-10 images are 32x32x3
        self.output = nn.Linear(512, 10)  # 10 classes for CIFAR-10
    
    def forward(self, x):
        x = self.flatten(x)
        x = torch.relu(self.hidden(x))
        x = self.output(x)
        return x

model = FullyConnectedNN()


In [4]:
criterion = nn.CrossEntropyLoss()  # Suitable for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
total_start_time = time.time()  # Record the total start time

for epoch in range(num_epochs):
    epoch_start_time = time.time()  # Record the start time for this epoch
    running_loss = 0.0
    
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    # Calculate and print the duration for the epoch
    epoch_end_time = time.time()
    epoch_duration = epoch_end_time - epoch_start_time
    if (epoch + 1) % 1 == 0:
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(trainloader):.4f}, Epoch Time: {epoch_duration:.2f} seconds")
        model.eval()
        correct = 0
        total = 0

        with torch.no_grad():
            for images, labels in testloader:
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network: {100 * correct / total:.2f}%')
# Calculate and print the total training duration
total_end_time = time.time()
total_duration = total_end_time - total_start_time
print(f"Total Training Time: {total_duration:.2f} seconds")


print('Finished Training')


Epoch 1/10, Loss: 1.6451, Epoch Time: 27.94 seconds



KeyboardInterrupt



In [6]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, f1_score

# Transformations for data normalization
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainloader = DataLoader(trainset, batch_size=100, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Files already downloaded and verified
Files already downloaded and verified


In [7]:
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(32 * 32 * 3, 512),
    nn.ReLU(),
    nn.Linear(512, 256),  # Additional hidden layer 1
    nn.ReLU(),
    nn.Linear(256, 128),  # Additional hidden layer 2
    nn.ReLU(),
    nn.Linear(128, 10)
).to(device)

In [ ]:
'''criterion = nn.CrossEntropyLoss()  # Suitable for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
total_start_time = time.time()  # Record the total start time

for epoch in range(num_epochs):
    epoch_start_time = time.time()  # Record the start time for this epoch
    running_loss = 0.0
    
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    # Calculate and print the duration for the epoch
    epoch_end_time = time.time()
    epoch_duration = epoch_end_time - epoch_start_time
    if (epoch + 1) % 1 == 0:
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(trainloader):.4f}, Epoch Time: {epoch_duration:.2f} seconds")
        model.eval()
        correct = 0
        total = 0

        with torch.no_grad():
            for images, labels in testloader:
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network: {100 * correct / total:.2f}%')
# Calculate and print the total training duration
total_end_time = time.time()
total_duration = total_end_time - total_start_time
print(f"Total Training Time: {total_duration:.2f} seconds")


print('Finished Training')'''

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 300
total_start_time = time.time()

for epoch in range(num_epochs):
    epoch_start_time = time.time()
    running_loss = 0.0
    
    model.train()
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    training_loss = running_loss / len(trainloader)
    
    # Validation phase
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    validation_loss = val_loss / len(testloader)
    accuracy = 100 * correct / total

    epoch_end_time = time.time()
    epoch_duration = epoch_end_time - epoch_start_time
    
    if (epoch + 1) % 30 == 0:
        # Show training and validation loss to check for overfitting
        print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {training_loss:.4f}, Validation Loss: {validation_loss:.4f}, Accuracy: {accuracy:.2f}%, Epoch Time: {epoch_duration:.2f} seconds')

total_end_time = time.time()
total_duration = total_end_time - total_start_time
print(f'Total Training Time: {total_duration:.2f} seconds')

print('Finished Training')

Epoch [30/300], Training Loss: 0.2116, Validation Loss: 3.2289, Accuracy: 53.16%, Epoch Time: 35.09 seconds
Epoch [60/300], Training Loss: 0.1413, Validation Loss: 4.6923, Accuracy: 52.73%, Epoch Time: 34.91 seconds
Epoch [90/300], Training Loss: 0.0854, Validation Loss: 5.5782, Accuracy: 52.67%, Epoch Time: 35.23 seconds
Epoch [120/300], Training Loss: 0.0809, Validation Loss: 6.2655, Accuracy: 53.38%, Epoch Time: 34.94 seconds
Epoch [150/300], Training Loss: 0.1127, Validation Loss: 6.5815, Accuracy: 52.69%, Epoch Time: 35.65 seconds
Epoch [180/300], Training Loss: 0.0712, Validation Loss: 7.6922, Accuracy: 53.09%, Epoch Time: 36.20 seconds
Epoch [210/300], Training Loss: 0.0766, Validation Loss: 8.4197, Accuracy: 52.43%, Epoch Time: 36.59 seconds
Epoch [240/300], Training Loss: 0.0674, Validation Loss: 9.4571, Accuracy: 52.93%, Epoch Time: 37.33 seconds
Epoch [270/300], Training Loss: 0.0697, Validation Loss: 10.3845, Accuracy: 52.91%, Epoch Time: 38.21 seconds
Epoch [300/300], Trai